<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
100% 84.9M/84.9M [00:00<00:00, 182MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [1]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [2]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [4]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [5]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [6]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [7]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [8]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [9]:
word2vec = api.load("glove-wiki-gigaword-300") #"word2vec-google-news-300"

In [10]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [11]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [12]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

  

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation+'@' for c in w) and len(w) > 3 and (w in self.word2vec)] # добавим к символам пунктуации симвл "@", чтобы убрать ники пользователей 
        return filtered_line

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        if len(tokens) == 0:
          return np.zeros_like(self.get_embeddings_('next'))
        embeddings = np.array([(w - self.mean)/self.std for w in self.word2vec[tokens]])
        return embeddings


        #if len(embeddings) == 0:
           # embeddings = np.zeros((1, self.word2vec.vector_size))
       # else:
          #  embeddings = np.array(embeddings)
          #  if len(embeddings.shape) == 1:
              #  embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [27]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [28]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [29]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
pca.fit(examples['features'])
examples["transformed_features"] = pca.transform(examples['features']) # Обучи PCA на эмбеддингах слов

In [30]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [31]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [32]:
from torch.utils.data import DataLoader


In [33]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Определим функции для тренировки и теста модели:

In [34]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        pred_targets = model(features)# Получи предсказания модели
        loss = criterion(pred_targets, targets) # Посчитай лосс
        # Обнови параметры модели
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")



def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            pred_targets = model(features)
            loss = criterion(pred_targets, targets) # Посчитай лосс
            preds = torch.argmax(pred_targets, dim=1)
            acc = (preds==targets).cpu().numpy().mean() # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [15]:
import torch.nn as nn
from torch.optim import Adam

In [36]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1

model = nn.Sequential(nn.Linear(vector_size, 1024), nn.ReLU(), nn.Linear(1024, num_classes))# Твоя модель
model = model.cuda()
criterion = nn.CrossEntropyLoss()# Твой лосс
optimizer = Adam(model.parameters(), lr )# Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [37]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5429359939098358, 'Test Acc': 0.729140625}


In [38]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5444025133554928, 'Test Acc': 0.7296544279153354}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [39]:
class TwitterDatasetForUnknownWords(TwitterDataset):

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation+'@' for c in w) and len(w) > 3]
        return filtered_line

    # версия, когда в качестве ембединга неизвестного слова берется контекст с шириной окна, равного 1
    def get_embeddings_(self, tokens):
        if len(tokens) == 0:
          return np.zeros_like(self.get_embeddings_('next'))
        embeddings = []
        for i, w in enumerate(tokens):
          if w in self.word2vec:
            embeddings.append((self.word2vec[w] - self.mean)/self.std)
          else:
            if i == 0:
              if len(tokens) > 1 and tokens[i+1] in self.word2vec:
                embeddings.append((self.word2vec[tokens[i+1]] - self.mean)/self.std)
              else:
                embeddings.append(np.zeros_like(self.word2vec['next']))
            else:
              if i == len(tokens) - 1:
                embeddings.append(embeddings[i-1])
              else:
                if tokens[i+1] in self.word2vec:
                  embeddings.append(embeddings[i-1] + (self.word2vec[tokens[i+1]] - self.mean)/self.std)
                else:
                  embeddings.append(embeddings[i-1])
                
        embeddings = np.array(embeddings)
        return embeddings  


In [40]:
dev = TwitterDatasetForUnknownWords(dev_data, "text", "emotion", word2vec)

In [41]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1

model = nn.Sequential(nn.Linear(vector_size, 1024), nn.ReLU(), nn.Linear(1024, num_classes))
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr )

In [42]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"] 

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5450632212162018, 'Test Acc': 0.72519921875}


In [43]:
print(testing(model, criterion, test_loader, device=device)) 

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'Test Loss': 0.5460617658429252, 'Test Acc': 0.7246124950079872}


# Версия TF-IDF + Word2Vec

In [44]:
!pip install sparsesvd

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sparsesvd import sparsesvd

vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, min_df=5)
filtered_dev_data = []
for line in dev_data['text']:
    line = " ".join(tokenizer.tokenize(line.lower()))
    new_line = ' '.join([w for w in line.split() if  len(w) > 3 and all(c not in string.punctuation+'@' for c in w) ]) 
    if new_line:
        filtered_dev_data.append(new_line)
filtered_dev_data = np.array(filtered_dev_data)
print(filtered_dev_data.shape)
X = vectorizer.fit_transform(filtered_dev_data)

(1277797,)


In [46]:
u, s, vt = sparsesvd(X.tocsc(), 300)
k, v = vectorizer.get_feature_names(), vt.T
tf_idf_embeddings = {k[i]:v[i] for i in range(len(v))}

In [47]:
class TwitterDatasetWithTfIdf(TwitterDataset):

    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tf_idf_embeddings, vectorizer):
        super().__init__(data, feature_column, target_column, word2vec)
        self.vectorizer = vectorizer
        self.tf_idf_embeddings = tf_idf_embeddings
        
    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation+'@' for c in w) and len(w) > 3]
        return filtered_line

    def get_embeddings_(self, tokens):
        if len(tokens) == 0:
          return np.zeros_like(self.get_embeddings_('next'))
        embeddings = []
        for w in tokens:   
          if w in self.word2vec:
            if w in self.tf_idf_embeddings.keys():
              embeddings.append((self.word2vec[w] - self.mean)/self.std + self.tf_idf_embeddings[w]) 
            else:
              embeddings.append((self.word2vec[w] - self.mean)/self.std)
          else:
            if w in self.tf_idf_embeddings.keys():
              embeddings.append(self.tf_idf_embeddings[w])
            else:
              embeddings.append(np.zeros(300))
                
        embeddings = np.array(embeddings)
        return embeddings

In [48]:
dev = TwitterDatasetWithTfIdf(dev_data, "text", "emotion", word2vec, tf_idf_embeddings, vectorizer)

In [49]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1

model = nn.Sequential(nn.Linear(vector_size, 1024), nn.ReLU(), nn.Linear(1024, num_classes))
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr )

In [50]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5555121970176696, 'Test Acc': 0.72711328125}


In [51]:
print(testing(model, criterion, test_loader, device=device)) 

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'Test Loss': 0.5539057738483905, 'Test Acc': 0.7280101837060703}


# Вывод
Как видим - Test Acc примерно одинаковый во всех 3 подходах и равен 72-73%. На мой взгляд, это может объясняться тем, что слова, которые встречаются в нашем тексте и есть в word2vec достаточно хорошо предобучены и достаточно точно описывают предложение, в котором они встречаются, но мы сталкиваемся с другой проблемой - в наших текстах очень много слов, которых нет в word2vec. Ниже я посчитал отношение количества слов, которые встречаются в тексте и есть в word2vec к количеству слов, которых встречаются в тексте и нет в word2vec, и получил неожиданный для себя результат - это отношение равняется $0.18$. Таким образом, естественно напрашивается вывод, что в случае, когда мы пытаемся сделать embedding'и слов с помощью tf-idf, для слов, которых нет в word2vec, мы получим практически нулевые векторы, то есть не передающие никакого смысла, а следовательно практические не влияющие на обучение модели. В случае, когда мы пытаемся получить вектор слов, которых нет в word2vec, суммируя embedding'и слов, которые окружают его, мы впринципе не сильно меняем ситуацию, когда используем только слова из word2vec, потому что таким образом как бы "удваиваем" контекст предложения.

In [94]:
words_word2vec = set() # слова, которые встретились в нашем тексте и есть в word2vec
for line in dev_data['text']:
    line = " ".join(tokenizer.tokenize(line.lower()))
    for w in line.split():
      if len(w)>3 and (w in word2vec) and all(c not in string.punctuation+'@' for c in w):
        words_word2vec.add(w)
    
words_not_in_word2vec = set() # слова, которые встретились в нашем тексте, но их нет в word2vec
for line in dev_data['text']:
    line = " ".join(tokenizer.tokenize(line.lower()))
    for w in line.split():
      if len(w)>3 and (w not in word2vec) and all(c not in string.punctuation+'@' for c in w):
        words_not_in_word2vec.add(w)

print(f'Количество слов, которые встретились в нашем тексте и есть в word2vec : {len(words_word2vec)} \nКоличество слов, которые встретились в нашем тексте, но их нет в word2vec : {len(words_not_in_word2vec)}')
print(f'Также посмотрим на соотношение между words_word2vec и words_not_in_word2vec : {len(words_word2vec)/len(words_not_in_word2vec)}')

Количество слов, которые встретились в нашем тексте и есть в word2vec : 83750 
Количество слов, которые встретились в нашем тексте, но их нет в word2vec : 463155
Также посмотрим на соотношение между words_word2vec и words_not_in_word2vec : 0.18082499379257483
